## 장르 기반 유사도로 추천

In [1]:
import pandas as pd
import numpy as np


In [55]:
movies = pd.read_csv('./tmdbData.csv')
print(movies.shape)
movies.head(2)

(9990, 8)


,title,genres,id,overview,popularity,original_title,vote_average,vote_count
0,앤트맨과 와스프: 퀀텀매니아,"['액션', '모험', 'SF']",640146,"슈퍼히어로 파트너인 스캇 랭과 호프 반 다인, 호프의 부모 재닛 반 다인과 행크 핌...",10773.574,Ant-Man and the Wasp: Quantumania,6.5,1999
1,슈퍼 마리오 브라더스,"['애니메이션', '모험', '가족', '판타지', '코미디']",502356,따단-딴-따단-딴 ♫ 전 세계를 열광시킬 올 타임 슈퍼 어드벤처의 등장! 뉴욕의 평...,8786.695,The Super Mario Bros. Movie,7.5,1580


In [56]:
movies_df = movies[['id','title','genres','vote_average','vote_count','popularity','overview','original_title']]

In [57]:
# 장르값이 여러개가 들어있어서 literal_eval로 tuple로 변환
from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)


In [ ]:
# genres컬럼에서 장르명만, keywords컬럼에서 키워드만 리스트로 넣기
# 리스트 내 여러개의 딕셔너리의 'name'키에 해당하는 값을 찾아서 리스트 형태로 변환
# movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
#movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])
# movies_df.head(2)

In [59]:
from sklearn.feature_extraction.text import CountVectorizer

# 장르값들을 벡터화 하기위해 띄어쓰기로 구분되는 문장으로 변환.
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
movies_df.head(2)

,id,title,genres,vote_average,vote_count,popularity,overview,original_title,genres_literal
0,640146,앤트맨과 와스프: 퀀텀매니아,"[액션, 모험, SF]",6.5,1999,10773.574,"슈퍼히어로 파트너인 스캇 랭과 호프 반 다인, 호프의 부모 재닛 반 다인과 행크 핌...",Ant-Man and the Wasp: Quantumania,액션 모험 SF
1,502356,슈퍼 마리오 브라더스,"[애니메이션, 모험, 가족, 판타지, 코미디]",7.5,1580,8786.695,따단-딴-따단-딴 ♫ 전 세계를 열광시킬 올 타임 슈퍼 어드벤처의 등장! 뉴욕의 평...,The Super Mario Bros. Movie,애니메이션 모험 가족 판타지 코미디


In [60]:
# min_df로 지정한 값보다 작은 경우에는 무시 
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2)) # ngram 단어의 묶음을 1개부터 2개까지 설정(science Fiction같은 2개 묶음)
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
genre_mat.shape
#4803,276 피처벡터행렬

(9990, 320)

In [61]:
# 생성한 행렬에 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:5])

(9990, 9990)
[[1.         0.1490712  0.2        ... 0.         0.16903085 0.        ]
 [0.1490712  1.         0.2981424  ... 0.         0.12598816 0.        ]
 [0.2        0.2981424  1.         ... 0.         0.50709255 0.        ]
 [0.6        0.1490712  0.2        ... 0.         0.16903085 0.        ]
 [0.4472136  0.11111111 0.1490712  ... 0.         0.12598816 0.33333333]]


In [62]:
# 유사도가 가장 높은 순으로 정렬
genre_sim_sorted = genre_sim.argsort()[:,::-1]
# print(genre_sim_sorted[:1])
# 0번 레코드는 3494번 레코드가 유사도 가장 높고, 가장 낮은 레코드는 2401번이다.

In [10]:
def find_movie(df, sorted_index, title_name, top_list=10): # top_list : 상위 10개만 추천

    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    # genre_sim_sorted에서 유사도 순으로 top_list개 index추출
    similar_index = sorted_index[title_index, :(top_list)]
    
    # top_list index출력. (2차원->1차원으로)
    print(similar_index)
    similar_index = similar_index.reshape(-1)
    
    return df.iloc[similar_index]

In [71]:
movies_df[movies_df['title'] == '아바타']

,id,title,genres,vote_average,vote_count,popularity,overview,original_title,genres_literal,weighted_vote
125,19995,아바타,"[액션, 모험, 판타지, SF]",7.6,29002,292.914,"가까운 미래, 지구는 에너지 고갈 문제를 해결하기 위해 머나먼 행성 판도라에서 대체...",Avatar,액션 모험 판타지 SF,7.565547


In [ ]:
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]
# 평균평점순으로 정렬했더니 투표수를 보니 평가횟수가 1회

In [70]:
similar_movies = find_movie(movies_df, genre_sim_sorted, '아바타', 10)
similar_movies[['title', 'vote_average']]
# kids는 장르가 아예 다름, 평점이 0인 영화도 있음

[[4432  125  203 2406  514 5641 3043 1296  524  896]]


,title,vote_average
4432,스콜피온 킹 2: 전사의 부활,4.6
125,아바타,7.6
203,잭 스나이더의 저스티스 리그,8.2
2406,판타스틱 4,5.8
514,토르: 라그나로크,7.6
5641,파이널 판타지 VII: 어드벤트 칠드런,7.1
3043,드래곤볼 에볼루션,2.9
1296,엑스맨: 다크 피닉스,6.0
524,정글 크루즈,7.4
896,라스트 에어벤더,4.7


In [63]:
# 가중평점(Weighted Rating) = (v/(v+m)) * R + (m/(v+m)) * C
# v: 개별 영화에 평점을 투효한 횟수
# m: 평점을 부여하기 위한 최소 투표 횟수
# R: 개별 영화에 대한 평균 평점
# C: 전체 영화에 대한 평균 평점

# v = 'vote_count'
# R = 'vote_average'
# C = movies_df['vote_average'].mean()

# m = 값이 높으면 평점 투표 횟수가 많은 영화에 높은 가중평점을 부여. 
# 상위 60%에 해당하는 횟수를 기준

C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('C:', round(C,3), 'm:', round(m,3))

C: 6.278 m: 776.0


In [64]:
def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ((v/(v+m)) * R) + ((m/(v+m)) * C)

movies_df['weighted_vote'] = movies.apply(weighted_vote_average, axis=1)

In [65]:
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote',ascending=False)

,title,vote_average,weighted_vote,vote_count
462,쇼생크 탈출,8.7,8.623152,23682
458,대부,8.7,8.598960,17826
1069,쉰들러 리스트,8.6,8.478074,14003
758,대부 2,8.6,8.444162,10787
633,다크 나이트,8.5,8.443222,29594
...,...,...,...,...
6758,재난 영화,3.3,4.660924,922
1570,판타스틱 4,4.4,4.631567,5517
3280,배트맨 앤 로빈,4.3,4.592352,4474
5134,데스노트,4.2,4.563656,3658


In [66]:
def find_sim_movie(df, sorted_idx, title_name, top_list=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    #print(title_movie)
    # 장르 유사성이 높은 영화 top_list의 2배수만큼 후보선정
    similar_index = sorted_idx[title_index, :(top_list*2)]
    similar_index = similar_index.reshape(-1)
    
    similar_index = similar_index[similar_index != title_index] # 기존 영화 인덱스 제외
    
    return df.iloc[similar_index].sort_values('weighted_vote', ascending=False)[:top_list]



In [67]:
similar_movie = find_sim_movie(movies_df,genre_sim_sorted,'아바타',10)
similar_movie[['title','vote_average','weighted_vote']]

,title,vote_average,weighted_vote
203,잭 스나이더의 저스티스 리그,8.2,8.044159
514,토르: 라그나로크,7.6,7.548163
524,정글 크루즈,7.4,7.247957
5802,윌로우,7.0,6.771379
5641,파이널 판타지 VII: 어드벤트 칠드런,7.1,6.737939
834,토르: 다크 월드,6.5,6.489695
501,어메이징 스파이더맨 2,6.5,6.486435
2071,워크래프트: 전쟁의 서막,6.4,6.386355
173,강철의 연금술사 완결편: 최후의 연성,6.3,6.281458
512,아쿠아맨과 로스트 킹덤,0.0,6.277898


## Surprise (협업 시스템기반)
### 유저가 평점주고 안주고 분류

- Surprise 는 사용자 아이디, 아이템 아이디, 평점 데이터가 로우 레벨로 된 데이터 세트만 적용할 수 있다. 
- 그래서 데이터의 첫번째 컬럼을 사용자 아이디, 두번째 컬럼을 아이템 아이디, 세번째 컬럼을 평점으로 가정해 데이터를 로딩하고 네번째 컬럼부터는 로딩을 수행하지 않는다. 
    - 예로, user_id, item_id, rating, time_stamp 필드로 구분된 데이터라면 앞 3개 필드만 로딩하고 이후 time_stamp 필드는 로딩에서 제외
    - 따라서 반드시 데이터 세트의 컬럼 순서가 사용자 아이디, 아이템 아이디, 평점 순으로 돼 있어야 한다.

https://big-dream-world.tistory.com/70?category=470634

### 1. Surprise를 이용한 추천 시스템 구축

- http://sruprise.readthedocs.io/en/stable/ 에 정리되어있다
- 간단한 예제 : 추천데이터를 학습용과 테스트용 데이터세트로 분리한 뒤 SVD 행렬분해를 통해 잠재요인협업필터링 수행하기

In [19]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [20]:
data = Dataset.load_builtin('ml-100k')
# 수행시마다 동일하게 데이터를 분할하기 위해 random_state값 부여
trainset, testset = train_test_split(data, test_size=0.2, random_state=0)

- surprise는 자체적으로 로우레벨의 데이터를 칼럼 레벨의 데이터로 변경하므로 원본인 로우 레벨의 사용자-아이템 데이터를 데이터 세트로 적용해야한다.

In [21]:
# SVD로 잠재 요인 협업 필터링 수행
algo=SVD() # 알고리즘 객체 생성
algo.fit(trainset)

- surprise에서 추천을 예측하는 메서드는 test()와 predict()이다.
- test()는 사용자-아이템 평점 데이터 세트 전체에 대해서 추천을 예측하는 메서드
- predict()는 개별 사용자와 영화에 대한 추천 평점을 반환하는 메서드

In [22]:
# 테스트데이터 세트에 대해 추천 수행
predictions = algo.test(testset)
print('prediction type:', type(predictions), 'size:',len(predictions))
print('prediction 결과의 최초 5개 추출:')
predictions[:5]
# 개별 사용자 아이디(uid), 영화(아이템)아이디(iid), 실제 평점(r_ui)정보에 기반해 surprise의 추천 예측 평점(est)데이터를 튜플 형태로 갖고있다.
# was_impossible이 True면 예측값을 생성할 수 없는 데이터라는 의미이다.

prediction type: <class 'list'> size: 20000
prediction 결과의 최초 5개 추출:


[Prediction(uid='269', iid='17', r_ui=2.0, est=2.46368560230615, details={'was_impossible': False}),
 Prediction(uid='704', iid='382', r_ui=4.0, est=3.6198566609939444, details={'was_impossible': False}),
 Prediction(uid='829', iid='475', r_ui=4.0, est=4.214699151236109, details={'was_impossible': False}),
 Prediction(uid='747', iid='274', r_ui=4.0, est=3.7571869625501684, details={'was_impossible': False}),
 Prediction(uid='767', iid='98', r_ui=5.0, est=4.7789277327723125, details={'was_impossible': False})]

In [23]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('269', '17', 2.46368560230615),
 ('704', '382', 3.6198566609939444),
 ('829', '475', 4.214699151236109)]

In [24]:
# predict로 추천예측해보기
# 사용자 아이디, 아이템 아이디를 문자열로 입력 -> 추천 예측 평점을 est로 반환
uid=str(196)
iid=str(302)
pred=algo.predict(uid,iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.06   {'was_impossible': False}


In [25]:
accuracy.rmse(predictions)

RMSE: 0.9482


0.9481573362793058

### 02. OS파일 데이터를 surprise데이터 세트로 로딩

In [26]:
import pandas as pd
ratings=pd.read_csv('./data/ratings_small.csv')
# ratings_noh.csv파일로 언로드시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('./data/ratings_noh.csv', index=False, header=False)

- Reader 클래스의 주요 생성 파라미터
    - line_format(string) : 칼럼을 순서대로 나열. 입력된 문자열을 공백으로 분리해 칼럼으로 인식
    - sep(char) : 칼럼을 분리하는 분리자(문자), 디폴트는 '\t'
    - rating_scale(tuple,optional) : 평점 값의 최소~ 최대 설정. 디폴트는(1,5)

In [27]:
from surprise import Reader

# 컬럼의 분리문자는 콤마, 평점 단위는 0.5, 최대평점 5
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('./data/ratings_noh.csv', reader=reader)

In [28]:
# SVD 행렬분해기법을 이용해 추천 예측 수행
trainset, testset = train_test_split(data, test_size=0.2, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정
# 잠재요인 크기 K값을 나타내는 파라미터인 n_facctors 50으로 설정
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습하고 테스트 데이터로 평점 예측 후 rmse 평가
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8851


0.8850646057753477

### 03. DataFrame에서 surprsie 데이터 세트로 로딩
- 주의할 점은 사용자 아이디, 아이템 아이디, 평점 컬럼 순서를 지켜야 한다.

In [29]:
import pandas as pd
from surprise import Reader, Dataset
 
ratings = pd.read_csv('./data/ratings_small.csv')
reader = Reader(rating_scale=(0.5, 5.0))
 
# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=0)


In [30]:
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8851


0.8850646057753477

### 04. 교차검증과 하이퍼 파라미터 튜닝

- Surprise 는 교차 검증과 하이퍼 파라미터 튜닝을 위해 사이킷런과 유사한 cross_validate() 와 GridSearchCV 클래스를 제공한다.
- cross_validate()는 인자로 알고리즘 객체, 데이터, 성능 평가 방법, 폴드 데이터 세트 개수(cv)를 입력받아, 출력결과와 같이 폴드별 성능 평가 수치와 전체 폴드의 평균 성능 평가 수치를 함께 보여준다.

In [31]:
from surprise.model_selection import cross_validate
 
# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩
ratings = pd.read_csv('./data/ratings_small.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
 
algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
# 폴드별 성능평가수치와 전체 폴드의 평균 성능평가수치를 출력

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8966  0.9014  0.8884  0.8965  0.8885  0.8943  0.0051  
MAE (testset)     0.6901  0.6951  0.6843  0.6895  0.6832  0.6884  0.0043  
Fit time          1.05    1.06    1.06    1.03    1.06    1.05    0.01    
Test time         0.29    0.17    0.31    0.17    0.22    0.23    0.06    


{'test_rmse': array([0.8966345 , 0.90135445, 0.88842928, 0.89647544, 0.88848994]),
 'test_mae': array([0.69006133, 0.69508191, 0.6842963 , 0.68949352, 0.68324344]),
 'fit_time': (1.0480284690856934,
  1.0642704963684082,
  1.0552177429199219,
  1.030167579650879,
  1.0569953918457031),
 'test_time': (0.28812170028686523,
  0.17130446434020996,
  0.30765819549560547,
  0.1675417423248291,
  0.21860766410827637)}

- Surprise 는 GridSearchCV를 통해 교차 검증을 통한 하이퍼 파라미터 최적화를 수행할 수 있다. 
- 하이퍼 파라미터 최적화는 알고리즘 유형에 따라 다를 수 있지만, SVD의 경우 주로 SGD(점진적 하강 방식)의 반복 횟수를 지정하는 n_epochs와 SVD의 잠재 요인 K의 크기를 지정하는 n_factors를 튜닝한다. 

In [32]:
from surprise.model_selection import GridSearchCV
 
# 최적화할 파라미터들을 딕셔너리 형태로 지정.
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }
 
# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)
 
# CV가 3일때 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
# n_epochs:20, n_factors:50일때 3개 폴드의 검증 데이터 세트에서 최적 RMSE가 약 0.8994로 도출되었다.

0.8993211310834207
{'n_epochs': 20, 'n_factors': 50}


In [33]:
from surprise.dataset import DatasetAutoFolds
 
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성.
data_folds = DatasetAutoFolds(ratings_file='./data/ratings_noh.csv', reader=reader)
 
#전체 데이터를 학습데이터로 생성함.
trainset = data_folds.build_full_trainset()
 
 
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)
 
 
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('./data/movies.csv')
 
# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인.
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')
 
print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
40       42  Dead Presidents (1995)  Action|Crime|Drama


In [34]:
uid = str(9)
iid = str(42)
 
pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 2.64   {'was_impossible': False}


In [35]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
     
    # 모든 영화들의 movieId를 리스트로 생성.
    total_movies = movies['movieId'].tolist()
     
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
     
    return unseen_movies
 
unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 45 추천대상 영화수: 9080 전체 영화수: 9125


In [36]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
     
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
     
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
     
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
     
    return top_movie_preds
 
unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')
 
for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 45 추천대상 영화수: 9080 전체 영화수: 9125
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.400579409001147
Godfather, The (1972) : 4.370540045092638
It Happened One Night (1934) : 4.363849637541674
Casablanca (1942) : 4.336024874543918
Maltese Falcon, The (1941) : 4.321015160843752
All About Eve (1950) : 4.309431147405599
It's a Wonderful Life (1946) : 4.303661756312046
African Queen, The (1951) : 4.289780175135358
Cinema Paradiso (Nuovo cinema Paradiso) (1989) : 4.283687861376393
Modern Times (1936) : 4.271531364943996
